# 신림역 음식점 크롤링

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd
import missingno as msno
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
import time
from bs4 import BeautifulSoup

In [2]:
driver = webdriver.Chrome('chromedriver/chromedriver.exe')

In [3]:
driver.get('https://map.kakao.com/')

In [4]:
driver.close()

In [5]:
def time_for_web():
    driver.implicitly_wait(10)
    time.sleep(3)

In [8]:
driver = webdriver.Chrome('chromedriver/chromedriver.exe')
driver.implicitly_wait(3)

In [10]:
# 카카오 지도로 이동
driver.get('https://map.kakao.com/')
time_for_web()

# '음식점' 검색어 입력
driver.find_element_by_id('search.keyword.query').send_keys('음식점')
driver.implicitly_wait(3)

# 검색 클릭
try:
    driver.find_element_by_id('search.keyword.submit').click()
except:
    print('확인 필요')

# 더보기 클릭
driver.find_element_by_css_selector('#info\.search\.place\.more').click()

In [18]:
driver.close()

In [11]:
from tqdm import tqdm

restaurant = {
    'name': [], 
    'category': [], 
    'address': [],
    'score': [], 
    'eval_cnt': [], 
    'review_cnt': []
}

In [12]:
for pagenum in tqdm(range(1, 35)):
    try: 
        # 현재 페이지의 음식점 정보 크롤링
        page = pagenum % 5
        soup = driver.page_source
        b = BeautifulSoup(soup, 'lxml')
        for num in range(0, 15):
            restaurant['name'].append(b.find_all('a','link_name')[num].text)
            restaurant['category'].append(b.find_all('span', 'subcategory')[num].text)
            restaurant['address'].append(b.find_all('p', 'lot_number')[num].text)
            restaurant['score'].append(b.find_all('em', 'num')[num].text)
            restaurant['eval_cnt'].append(b.find_all('a', 'numberofscore')[num].text)
            restaurant['review_cnt'].append(b.find_all('a', 'review', 'em')[num].text)
        time_for_web
        # 다음 페이지 or 다음 버튼
        if page:
            driver.find_element_by_id('info.search.page.no{}'.format(page+1)).click()
        else:
            driver.find_element_by_id('info.search.page.next').click()
    except:
        print('페이지 초과 or 끝')
        break

 97%|███████████████████████████████████████████████████████████████████████████████▌  | 33/34 [00:59<00:01,  1.81s/it]

페이지 초과 or 끝


In [15]:
restaurant.keys()

dict_keys(['name', 'category', 'address', 'score', 'eval_cnt', 'review_cnt'])

In [17]:
df = pd.DataFrame(restaurant)
df

,name,category,address,score,eval_cnt,review_cnt
0,아리차이 신림점,중화요리,(지번) 신림동 1430-11,3.8,64건,리뷰 45
1,디자이너리카페,카페,(지번) 신림동 1465-12,3.6,33건,리뷰 184
2,원조민속순대타운,순대,(지번) 신림동 1640-31,3.3,30건,리뷰 175
3,월화고기 보라매직영점,"육류,고기",(지번) 봉천동 732-3,4.2,103건,리뷰 215
4,막불감동 신림본점,냉면,(지번) 신림동 1433-60,3.7,41건,리뷰 97
...,...,...,...,...,...,...
505,빽다방 신림역2호점,커피전문점,(지번) 신림동 1641-12,3.8,5건,리뷰 3
506,궁중 삼계탕,삼계탕,(지번) 신대방동 395-66,3.2,6건,리뷰 5
507,본죽 신림역점,죽,(지번) 신림동 1637-3,2.6,9건,리뷰 3
508,고고즉석떡볶이 신림역점,분식,(지번) 신림동 1424-9,4.5,4건,리뷰 13


In [18]:
df.shape

(510, 6)

In [22]:
df.to_csv('./source/restaurant_raw(mk).csv', index=False)

In [24]:
df = pd.read_csv('./source/restaurant_raw(mk).csv')
df

,name,category,address,score,eval_cnt,review_cnt
0,아리차이 신림점,중화요리,(지번) 신림동 1430-11,3.8,64건,리뷰 45
1,디자이너리카페,카페,(지번) 신림동 1465-12,3.6,33건,리뷰 184
2,원조민속순대타운,순대,(지번) 신림동 1640-31,3.3,30건,리뷰 175
3,월화고기 보라매직영점,"육류,고기",(지번) 봉천동 732-3,4.2,103건,리뷰 215
4,막불감동 신림본점,냉면,(지번) 신림동 1433-60,3.7,41건,리뷰 97
...,...,...,...,...,...,...
505,빽다방 신림역2호점,커피전문점,(지번) 신림동 1641-12,3.8,5건,리뷰 3
506,궁중 삼계탕,삼계탕,(지번) 신대방동 395-66,3.2,6건,리뷰 5
507,본죽 신림역점,죽,(지번) 신림동 1637-3,2.6,9건,리뷰 3
508,고고즉석떡볶이 신림역점,분식,(지번) 신림동 1424-9,4.5,4건,리뷰 13
